# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

### Resolução da atividade 1)

In [1]:
import pandas as pd
import requests

sinasc = pd.read_csv('SINASC_RO_2019.csv')
shape = sinasc.shape
print(f'Há um total de {shape[0]} linhas e {shape[1]} colunas.')

def count_dup(data):
    dp = data.drop_duplicates().shape
    if shape == dp:
        print(f'Após remover os valores duplicados o número de linhas permanece inalterado, ou seja não há dados duplicados.')

    else:
        print(f'Os dados duplicados foram removidos, agora o dataframe contém {dp[0]} linhas e {dp[1]} colunas.')
        
count_dup(sinasc)

Há um total de 27028 linhas e 69 colunas.
Após remover os valores duplicados o número de linhas permanece inalterado, ou seja não há dados duplicados.


### Resolução da atividade 2)

In [2]:
def count_nulls(dataFrame):
    nulls = dataFrame.isna().sum()
    print(nulls)

count_nulls(sinasc)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


### Resolução da atividade 3)

In [3]:
# Criando novo data frame apenas com as variáveis de interesse.
df = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
# Contagem dos valores nulos
count_nulls(df)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


### Resolução da atividade 4)

In [4]:
na = df['APGAR5'].isna().sum()
linhas = len(df)
print(f'Há um total de {linhas} linhas, onde {na} são valores missing.')

df_clean = df.copy()
df_clean.dropna(subset=['APGAR5'], inplace=True)
n_linha = len(df_clean)
print(f'Foram removidas as {na} linhas com dados faltantes na coluna APGAR5, agora a coluna possui {n_linha} linhas')

Há um total de 27028 linhas, onde 103 são valores missing.
Foram removidas as 103 linhas com dados faltantes na coluna APGAR5, agora a coluna possui 26925 linhas


### Resolução da atividade 5)

observe que as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9.

In [5]:
df_clean[['ESTCIVMAE', 'CONSULTAS']].isna().sum()


ESTCIVMAE    315
CONSULTAS      0
dtype: int64

In [6]:
df_clean[['ESTCIVMAE', 'CONSULTAS']].fillna(9)

,ESTCIVMAE,CONSULTAS
0,5.0,4
1,2.0,4
2,9.0,4
3,5.0,3
4,2.0,4
...,...,...
27023,2.0,4
27024,1.0,4
27025,2.0,4
27026,2.0,4


### Resolução da atividade 6)

In [14]:
na_values = df_clean['QTDFILVIVO'].isna().sum()
print(f'Há um total de {na_values} dados faltantes.')
df_clean['QTDFILVIVO'].fillna(0, inplace=True)

na_values1 = df_clean['QTDFILVIVO'].isna().sum()
if na_values1 < 0:
    print('Houve algum erro ao tentar substituir os valores nulos por 0! revise e tente novamente.')

else:
    print(f'Os {na_values} valores missing foram substituidos por 0')

Há um total de 1566 dados faltantes.
Os 1566 valores missing foram substituidos por 0


### Resolução da atividade 7)

In [16]:
df_clean

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [20]:
# Encontra as colunas com valores nulos
cols_com_missing = df_clean.columns[df_clean.isnull().any()]

# Itera pelas colunas com valores nulos e imprime a contagem
for col in cols_com_missing:
    qtd_missing = df_clean[col].isnull().sum()
    print(f'A variável "{col}" contém {qtd_missing} valores missing.')


A variável "ESTCIVMAE" contém 315 valores missing.
A variável "ESCMAE" contém 310 valores missing.
A variável "GESTACAO" contém 1216 valores missing.
A variável "GRAVIDEZ" contém 76 valores missing.


Como as variáveis 'GESTACAO', 'GRAVIDEZ', 'ESTCIVMAE', 'ESCMAE' são dados de certa forma pessoais, que variam de acordo com a pessoa, creio que o melhor seja preemcher os valores missing com 'não preenchido'

In [21]:
# Substitui os valores nulos por "não preenchido"
df_clean.fillna('não preenchido', inplace=True)

C:\Users\alima\AppData\Local\Temp\ipykernel_5048\585331849.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'não preenchido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_clean.fillna('não preenchido', inplace=True)


In [23]:
# Confere se ainda há variáveis como algum valor missing
df_clean.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

### Resolução da atividade 8)

In [26]:
# Criando nova categoria
df_clean['CAT_APGAR'] = ''

# Estrutura de para classificar de acordo com o valore de 'APGAR'
for index, row in df_clean.iterrows():
    if 0 <= row['APGAR5'] <= 3:
        df_clean.loc[index, 'CAT_APGAR'] = 'asfixia severa'
    elif 4 <= row['APGAR5'] <= 5: 
        df_clean.loc[index, 'CAT_APGAR'] = 'asfixia moderada'
    elif 6 <= row['APGAR5'] <= 7: 
        df_clean.loc[index, 'CAT_APGAR'] = 'asfixia leve'
    elif 8 <= row['APGAR5'] <= 10:
        df_clean.loc[index, 'CAT_APGAR'] = 'normal'

In [47]:
# Frequência que de cada nivel de APGAR5
freq_apgar = df_clean['CAT_APGAR'].value_counts()

perc_apgar = (freq_apgar / len(df_clean)) * 100

for cat, cont in freq_apgar.items():
    porcentagem = perc_apgar[cat]
    print(f"{cat:-<20} {porcentagem:.1f}% Total: {cont:->15}")

normal-------------- 98.3%  Total: ----------26463
asfixia leve-------- 1.2%  Total: ------------320
asfixia severa------ 0.3%  Total: -------------74
asfixia moderada---- 0.3%  Total: -------------68


### Resolução da atividade 9)

In [56]:
#Verifica o nome atual das colunas no DataFrame
df_clean.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'cat_apgar'],
      dtype='object')

In [54]:
df_clean.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao',
                   'gravidez', 'consultas', 'apgar5', 'cat_apgar']

In [57]:
#Visualiza o DataFrame com os novos nomes das calunas.
df_clean

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,cat_apgar
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
